In [3]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [4]:
start_page = 1 #search result page start
end_page = 1 #search results page end

URL_template = "https://www.property24.com/for-sale/cape-town/western-cape/432/p"

for pg in range(start_page, end_page+1):
    URL = URL_template + str(pg)
    page = requests.get(URL) #use requests library to get page
    soup = BeautifulSoup(page.content, "html.parser") #use BeautifulSoup library to get page content


    results = soup.find_all("div", {"itemtype":"http://schema.org/Product"}) #finds all the elements we want notation is ("html element", {element attribute: attribute value})

    def convert_find_soup_to_text(soup): #finds text if available, removes formats, then returns text
        if soup != None: soup = soup.text.replace("\n","").replace("\r", "").replace("\xa0", "").replace("m²","").strip()

        return soup 

    scraped_data = [["ID", "Link", "Title", "Location", "Excerpt", "Address", "Size (Meters Squared)", "Price (ZAR)", "Bedrooms", "Bathrooms", "Parking Spaces", "Estate Agency"]] #this will be the header of the table

    for element in results:
        try:
            link = element.find("a")["href"] #finds the attribute value of the html element. notation is list_item.find("html elemnt")["element attribute"]
            link = "https://www.property24.com" + link
        except:
            link = None

        try: 
            id = link.split("/")[-1]
        except:
            id = None

        information = ["p24_title", "p24_location", "p24_excerpt", "p24_address", "p24_size", "p24_price"]
        for info in range(len(information)):
            inf = element.find("span", {"class": information[info]})
            information[info] = convert_find_soup_to_text(inf)

        feature_details = ["Bedrooms", "Bathrooms", "Parking Spaces"]
        for detail in range(len(feature_details)):
            feature = element.find("span", {"title": feature_details[detail]})
            feature_details[detail] = convert_find_soup_to_text(feature)
        
        try:
            estate_agency = element.find("span", {"class":"p24_branding"})["title"]
            estate_agency = estate_agency.replace("Estate Agency profile for ", "")
        except:
            estate_agency = None

        row = [id] + [link] + information + feature_details + [estate_agency]
        scraped_data.append(row)

    data_base = pd.DataFrame(scraped_data) #converts each page results into a dataframe with pandas
    data_base.to_csv(f"scraped_data_page{pg}.csv", index=False, header=False) #saves each dataframe into a separate .csv
